In [ ]:
import numpy as np
import pandas as pd
import librosa as lib

## Importar los rttm y convertirlos en un df

In [ ]:

def read_rttm(file_path):
    # Definimos las columnas que esperamos en el archivo .rttm
    columns = ['Type', 'File ID', 'Channel', 'Start Time', 'Duration', 'Ortho', 'Ortho1', 'SType', 'Conf']
    
    # Leer el archivo y procesar cada línea
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
    # Procesar las líneas para crear una lista de listas (filas y columnas)
    data = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == len(columns):
            data.append(parts)
    
    # Crear el DataFrame
    df = pd.DataFrame(data, columns=columns)
    
    # Convertir columnas a los tipos de datos adecuados
    df['Start Time'] = df['Start Time'].astype(float)
    df['Duration'] = df['Duration'].astype(float)
    df['Conf'] = df['Conf'].astype(float)
    
    return df

# Usar la función para leer un archivo .rttm
file_path = 'alma-a1-nsb/alma-a1-nsb.rttm'
df_rttm = read_rttm(file_path)

# Cambiar todos los valores de la columna 'Type' a "Elan"
df_rttm['Type'] = 'Elan'
df_rttm= df_rttm[['Type', 'File ID','Start Time', 'Duration', 'SType']]

#Averiguar que tipos de speakers

df_rttm_stype_values= df_rttm["SType"].unique()
df_rttm_stype_values



### Creo diccionario para ajustar los speakers de la desgrabación con los de la diarización

In [ ]:
# Diccionario con las correspondencias
stype_dict = {
    'CHI': 'KCHI',
    'MOT': 'FEM',
    'FAT': 'MAL',
    'MUJ': 'FEM',
    'NIN': 'OCH'}

# Crear la nueva columna 'STypeNew' usando .map()
df_rttm['STypeNew'] = df_rttm['SType'].map(stype_dict)
df_rttm

In [ ]:
def read_rttm_diar(file_path):
    # Definimos las columnas que observamos en el archivo
    columns = ['Type', 'File ID', 'Channel', 'Start Time', 'Duration', 'Ortho1', 'Ortho2', 'SType', 'Name1', 'Name2']
    
    # Leer el archivo y procesar cada línea
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
    # Procesar las líneas para crear una lista de listas (filas y columnas)
    data = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == len(columns):
            data.append(parts)
    
    # Crear el DataFrame
    df = pd.DataFrame(data, columns=columns)
    
    # Convertir columnas a los tipos de datos adecuados
    df['Start Time'] = df['Start Time'].astype(float)
    df['Duration'] = df['Duration'].astype(float)
    
    return df

# Leer el archivo .rttm actualizado
file_path = 'alma-a1-nsb-diarization.rttm'
df_rttm_diar = read_rttm_diar(file_path)

# Cambiar todos los valores de la columna 'Type' a "Diarization"
df_rttm_diar['Type'] = 'Diarization'

df_rttm_diar=df_rttm_diar[['Type', 'File ID','Start Time', 'Duration', 'SType']]

df_rttm_diar['STypeNew']=df_rttm_diar['SType']
df_rttm_diar

In [ ]:
# Junto ambos df en uno solo 
df_both = pd.concat([df_rttm, df_rttm_diar])
df_both 

In [ ]:
# Filtro en tiempo para que todas las intervenciones queden dentro de las dos horas del medio (3600s < Start time < 10800)
df_both_filtered = df_both.query('3600 < `Start Time` < 10800')


In [ ]:
import numpy as np

# Define tus variables
start = 3600
end = 10800
step = 0.01

# Crear listas vacías
Elan_KCHI = []
Elan_OCH = []
Elan_FEM = []
Elan_MAL = []

Diar_KCHI = []
Diar_OCH = []
Diar_FEM = []
Diar_MAL = []

# Recorrer el DataFrame con cada valor de i
for i in np.arange(start, end, step):
    # Variables para controlar si se encontró un match en las condiciones
    elan_kchi_match = False
    elan_och_match = False
    elan_fem_match = False
    elan_mal_match = False
    
    diar_kchi_match = False
    diar_och_match = False
    diar_fem_match = False
    diar_mal_match = False
    
    # Iterar sobre cada fila del DataFrame
    for index, row in df_both_filtered.iterrows():
        # Verificar si i está en el intervalo de Start Time y Duration
        if row['Start Time'] <= i < (row['Start Time'] + row['Duration']):
            # Agregar a la lista correspondiente dependiendo de STypeNew y Type
            if row['STypeNew'] == 'KCHI':
                if row['Type'] == 'Elan':
                    elan_kchi_match = True
                elif row['Type'] == 'Diarization':
                    diar_kchi_match = True
            elif row['STypeNew'] == 'OCH':
                if row['Type'] == 'Elan':
                    elan_och_match = True
                elif row['Type'] == 'Diarization':
                    diar_och_match = True
            elif row['STypeNew'] == 'FEM':
                if row['Type'] == 'Elan':
                    elan_fem_match = True
                elif row['Type'] == 'Diarization':
                    diar_fem_match = True
            elif row['STypeNew'] == 'MAL':
                if row['Type'] == 'Elan':
                    elan_mal_match = True
                elif row['Type'] == 'Diarization':
                    diar_mal_match = True
    
    # Append 1 o 0 dependiendo si hubo match
    Elan_KCHI.append(1 if elan_kchi_match else 0)
    Elan_OCH.append(1 if elan_och_match else 0)
    Elan_FEM.append(1 if elan_fem_match else 0)
    Elan_MAL.append(1 if elan_mal_match else 0)
    
    Diar_KCHI.append(1 if diar_kchi_match else 0)
    Diar_OCH.append(1 if diar_och_match else 0)
    Diar_FEM.append(1 if diar_fem_match else 0)
    Diar_MAL.append(1 if diar_mal_match else 0)


In [ ]:
# Elan_KCHI = []
# Elan_OCH = []
# Elan_FEM = []
# Elan_MAL = []

# Diar_KCHI = []
# Diar_OCH = []
# Diar_FEM = []
print(Diar_MAL)
